# Merging DataFrames

In [ ]:
import numpy as np
import pandas as pd

## Our Dataset
- Our datasets are spread across multiple files in this section. Each file has a `restaurant_` prefix.
- The `customers.csv` file stores our restaurant's customers.
- The `foods.csv` file stores our restaurant's menu items.
- The `week_1_sales` and `week_2_sales` files store our orders.

In [ ]:
food = pd.read_csv('restaurant_foods.csv')
customers = pd.read_csv('restaurant_customers.csv')
week1 = pd.read_csv('restaurant_week_1_sales.csv')
week2 = pd.read_csv('restaurant_week_2_sales.csv')

## The pd.concat Function I
- The `concat` function concatenates one **DataFrame** to the end of another.
- The original index labels will be kept by default. Set `ignore_index` to True to generate a new index.
- The `keys` parameter create a **MultiIndex** using the specified keys/labels.

In [ ]:
pd.concat([week1, week2], ignore_index=False)

In [ ]:
pd.concat([week1, week2], ignore_index=True)

In [ ]:
pd.concat([week1, week2], keys=['First week', 'Second week'])

## The pd.concat Function II
- Pandas will concatenate the **DataFrames** along the row/index axis.
- Pandas will include all columns that exist in either **DataFrame**. If there are no matching values, pandas will use `NaN` values.
- We can pass the `axis` parameter an argument of `"columns"` to concatenate on the column axis.

In [ ]:
df1 = pd.DataFrame(data=[1, 2, 3], columns=['A'])
df1

In [ ]:
df2 = pd.DataFrame(data=[4, 5, 6], columns=['B'])
df2

In [ ]:
pd.concat([df1, df2])

In [ ]:
pd.concat([df1, df2], axis='index')

In [ ]:
pd.concat([df1, df2], axis='columns')

## Left Joins
- The `merge` method joins two **DataFrames** together based on shared values in a column or an index.
- A left join merges one **DataFrame** into another based on values in the first one.
- The "left" **DataFrame** is the one we invoke the `merge` method on.
- If the left **DataFrame's** value is not found in the right **DataFrame**, the row will hold `NaN` values.
<img src="SQL_Joins.png" width="800" height="800"/>

In [ ]:
week1.merge(right=food, how='left', on='Food ID')

## The left_on and right_on Parameters
- The `left_on` and `right_on` parameters designate the column names from each **DataFrame** to use in the merge.

In [ ]:
week1

In [ ]:
customers

In [ ]:
week1.merge(right=customers, how='left', left_on='Customer ID', right_on='ID')

In [ ]:
week1.merge(right=customers, how='left', left_on='Customer ID', right_on='ID').drop('ID', axis='columns')
week1.merge(right=customers, how='left', left_on='Customer ID', right_on='ID').drop('ID', axis=1)

## Inner Joins I
- Inner joins merge two tables based on *shared*/*common* values in columns.
- If only one **DataFrame** has a value, pandas will exclude it from the final results set.
- If the same ID occurs multiple times, pandas will store each possible combination of the values.
- The design of the join ensures that the results will be the same no matter what **DataFrame** the `merge` method is invoked upon.
<img src="SQL_Joins.png" width="800" height="800"/>

In [ ]:
week1

In [ ]:
week2

In [ ]:
week1.merge(right=week1, how='inner', on='Customer ID')

In [ ]:
week1.merge(right=week1, how='inner', on='Customer ID', suffixes=(' - Week 1', ' - Week 2'))

## Inner Joins II
- We can pass multiple arguments to the `on` parameter of the `merge` method. Pandas will require matches in both columns across the **DataFrames**.

In [ ]:
week1

In [ ]:
week2

In [ ]:
week1.merge(right=week2, how='inner', on=['Customer ID', 'Food ID'])

## Full/Outer Join
- A **full/outer** joins values that are found in either **DataFrame** or both **DataFrames**.
- Pandas does not mind if a value exists in one **DataFrame** but not the other.
- If a value does not exist in one **DataFrame**, it will have a `NaN`.

<img src="SQL_Joins.png" width="800" height="800"/>

In [41]:
week1

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


In [42]:
week2

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [46]:
merged = week1.merge(right=week2, how='outer', on='Customer ID', suffixes=(' - Week 1', ' - Week 2'), indicator=True)
merged

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [57]:
merged.rename(columns={'_merge': 'Merge'}, inplace=True)
merged

,Customer ID,Food ID - Week 1,Food ID - Week 2,Merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [59]:
merged[merged['Merge'].isin(['left_only', 'right_only'])]

,Customer ID,Food ID - Week 1,Food ID - Week 2,Merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [61]:
merged[merged['Merge'] == 'both']

,Customer ID,Food ID - Week 1,Food ID - Week 2,Merge
5,21,4.0,4.0,both
6,21,4.0,4.0,both
10,30,2.0,4.0,both
14,45,7.0,8.0,both
35,75,6.0,4.0,both
...,...,...,...,...
424,922,1.0,2.0,both
430,937,10.0,10.0,both
431,937,10.0,2.0,both
436,945,5.0,4.0,both


## Merging by Indexes with the left_index and right_index Parameters
- Use the `on` parameter if the column(s) to be matched on have the same names in both **DataFrames**.
- Use the `left_on` and `right_on` parameters if the column(s) to be matched on have different names in the two **DataFrames**.
- Use the `left_index` or `right_index` parameters (set to True) to if the values to be matched on are found in the index of a **DataFrame**.

## The join Method
- The `join` method is a shortcut for concatenating two **DataFrames** when merging by index labels.